In [1]:
import cdsapi
import xarray as xr
import rioxarray
import configparser as cfg
import yaml
from tqdm import tqdm

In [2]:
def add_var(data, var):
    df = xr.open_dataset('download.nc')
    data_vars_list = [v for v in df.data_vars.keys()]
    data_vars = ",".join(data_vars_list)
    units = ",".join([df[v].attrs['units'] for v in data_vars_list])
    long_names = ",".join([df[v].attrs['long_name'] for v in data_vars_list])
    data[var] = {
        "factor": 1,
        "original_units": units,
        "convert_units": units,
        "cams_variables": data_vars,
        "long_names": long_names
    }

In [3]:
data = {}
skipped = []
c = cdsapi.Client()

In [4]:
single_level_vars = [
    '10m_u_component_of_wind',
    '10m_v_component_of_wind',
    '2m_dewpoint_temperature',
    '2m_temperature',
    'black_carbon_aerosol_optical_depth_550nm',
    'dust_aerosol_optical_depth_550nm',
    'land_sea_mask',
    'mean_sea_level_pressure',
    'organic_matter_aerosol_optical_depth_550nm',
    'particulate_matter_10um',
    'particulate_matter_1um',
    'particulate_matter_2.5um',
    'sea_salt_aerosol_optical_depth_550nm',
    'sulphate_aerosol_optical_depth_550nm',
    'surface_geopotential',
    'surface_pressure',
    'total_aerosol_optical_depth_1240nm',
    'total_aerosol_optical_depth_469nm',
    'total_aerosol_optical_depth_550nm',
    'total_aerosol_optical_depth_670nm',
    'total_aerosol_optical_depth_865nm',
    'total_column_carbon_monoxide',
    'total_column_ethane',
    'total_column_formaldehyde',
    'total_column_hydrogen_peroxide',
    'total_column_hydroxyl_radical',
    'total_column_isoprene',
    'total_column_methane',
    'total_column_nitric_acid',
    'total_column_nitrogen_dioxide',
    'total_column_nitrogen_monoxide',
    'total_column_ozone',
    'total_column_peroxyacetyl_nitrate',
    'total_column_propane',
    'total_column_sulphur_dioxide',
    'total_column_water_vapour',
]
multi_level_vars = [
    'carbon_monoxide',
    'dust_aerosol_0.03-0.55um_mixing_ratio',
    'dust_aerosol_0.55-0.9um_mixing_ratio',
    'dust_aerosol_0.9-20um_mixing_ratio',
    'ethane',
    'formaldehyde',
    'hydrogen_peroxide',
    'hydrophilic_black_carbon_aerosol_mixing_ratio',
    'hydrophilic_organic_matter_aerosol_mixing_ratio',
    'hydrophobic_black_carbon_aerosol_mixing_ratio',
    'hydrophobic_organic_matter_aerosol_mixing_ratio',
    'hydroxyl_radical',
    'isoprene',
    'nitric_acid',
    'nitrogen_dioxide',
    'nitrogen_monoxide',
    'ozone',
    'peroxyacetyl_nitrate',
    'propane',
    'sea_salt_aerosol_0.03-0.5um_mixing_ratio',
    'sea_salt_aerosol_0.5-5um_mixing_ratio',
    'sea_salt_aerosol_5-20um_mixing_ratio',
    'specific_humidity',
    'sulphate_aerosol_mixing_ratio',
    'sulphur_dioxide',
    'temperature',
]
single_slow_access = [
    'high_cloud_cover',
    'high_vegetation_cover',
    'lake_cover',
    'leaf_area_index_high_vegetation',
    'leaf_area_index_low_vegetation',
    'lifting_threshold_speed',
    'low_cloud_cover',
    'low_vegetation_cover',
    'mean_altitude_of_maximum_injection',
    'medium_cloud_cover',
    'near_ir_albedo_for_diffuse_radiation',
    'near_ir_albedo_for_direct_radiation',
    'sea_ice_cover',
    'sea_surface_temperature',
    'skin_reservoir_content',
    'skin_temperature',
    'snow_albedo',
    'snow_depth',
    'soil_clay_content',
    'soil_type',
    'surface_roughness',
    'total_cloud_cover',
    'total_column_acetone',
    'total_column_aldehydes',
    'total_column_ethanol',
    'total_column_ethene',
    'total_column_formic_acid',
    'total_column_methanol',
    'total_column_methyl_peroxide',
    'total_column_olefins',
    'total_column_organic_nitrates',
    'total_column_paraffins',
    'total_column_water',
    'type_of_high_vegetation',
    'type_of_low_vegetation',
    'uv_visible_albedo_for_diffuse_radiation',
    'uv_visible_albedo_for_direct_radiation',
    'vertically_integrated_mass_of_dust_aerosol_0.03-0.55um',
    'vertically_integrated_mass_of_dust_aerosol_0.55-9um',
    'vertically_integrated_mass_of_dust_aerosol_9-20um',
    'vertically_integrated_mass_of_hydrophilic_black_carbon_aerosol',
    'vertically_integrated_mass_of_hydrophilic_organic_matter_aerosol',
    'vertically_integrated_mass_of_hydrophobic_black_carbon_aerosol',
    'vertically_integrated_mass_of_hydrophobic_organic_matter_aerosol',
    'vertically_integrated_mass_of_sea_salt_aerosol_0.03-0.5um',
    'vertically_integrated_mass_of_sea_salt_aerosol_0.5-5um',
    'vertically_integrated_mass_of_sea_salt_aerosol_5-20um',
    'vertically_integrated_mass_of_sulphate_aerosol'
]
multi_slow_access = [
    'acetone',
    'acetone_product',
    'aldehydes',
    'amine',
    'ammonia',
    'ammonium',
    'dimethyl_sulfide',
    'dinitrogen_pentoxide',
    'ethanol',
    'ethene',
    'formic_acid',
    'fraction_of_cloud_cover',
    'geopotential',
    'hydroperoxy_radical',
    'lead',
    'methacrolein_mvk',
    'methacrylic_acid',
    'methane_chemistry',
    'methane_sulfonic_acid',
    'methanol',
    'methyl_glyoxal',
    'methyl_peroxide',
    'methylperoxy_radical',
    'nitrate',
    'nitrate_radical',
    'olefins',
    'organic_ethers',
    'organic_nitrates',
    'paraffins',
    'pernitric_acid',
    'peroxides',
    'peroxy_acetyl_radical',
    'potential_vorticity',
    'propene',
    'radon',
    'relative_humidity',
    'specific_cloud_ice_water_content',
    'specific_cloud_liquid_water_content',
    'specific_rain_water_content',
    'specific_snow_water_content',
    'stratospheric_ozone_tracer',
    'terpenes',
    'u_component_of_wind',
    'v_component_of_wind',
    'vertical_velocity',
]

In [5]:
for var in tqdm(single_level_vars):
    c.retrieve(
    'cams-global-reanalysis-eac4',
    {
        'format': 'netcdf',
        'variable': var,
        'date': '2010-01-01/2010-01-02',
        'time': '00:00',
        'area': [
            90, 179, 89,
            180,
        ],
    },
    'download.nc')
    add_var(data, var)
with open("eac4_config.yaml", "w") as file:
    yaml.dump(data, file, sort_keys=False)
data = {}


  0%|          | 0/36 [00:00<?, ?it/s]2023-07-15 15:28:21,856 INFO Welcome to the CDS
2023-07-15 15:28:21,857 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:21,957 INFO Request is completed
2023-07-15 15:28:21,959 INFO Downloading https://download-0005-ads-clone.copernicus-climate.eu/cache-compute-0005/cache/data2/adaptor.mars.internal-1689418070.95915-1406-15-d5427b34-2b0a-40d0-a2e8-9ac04d978288.nc to download.nc (1.1K)
2023-07-15 15:28:22,084 INFO Download rate 8.8K/s
  3%|▎         | 1/36 [00:00<00:20,  1.72it/s]2023-07-15 15:28:22,286 INFO Welcome to the CDS
2023-07-15 15:28:22,287 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:22,357 INFO Downloading https://download-0002-ads-clone.copernicus-climate.eu/cache-compute-0002/cache/data2/adaptor.mars.internal-1689418074.0917246-15188-14-086bf3e6-06d3-48d4-85e2-ae992de11b32.nc to downlo

In [6]:
for var in tqdm(multi_level_vars):
    c.retrieve(
    'cams-global-reanalysis-eac4',
    {
        'format': 'netcdf',
        'variable': var,
        'date': '2010-01-01/2010-01-02',
        'time': '00:00',
        "model_level": '1',
        'area': [
            90, 179, 89,
            180,
        ],
    },
    'download.nc')
    add_var(data, var)
with open("eac4_config.yaml", "a") as file:
    yaml.dump(data, file, sort_keys=False)
data = {}

  0%|          | 0/26 [00:00<?, ?it/s]2023-07-15 15:28:31,386 INFO Welcome to the CDS
2023-07-15 15:28:31,387 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:31,430 INFO Downloading https://download-0005-ads-clone.copernicus-climate.eu/cache-compute-0005/cache/data3/adaptor.mars.internal-1689418443.7025523-4414-12-eb528201-d47c-481a-890c-ed9ca1f4bad7.nc to download.nc (1.2K)
2023-07-15 15:28:31,499 INFO Download rate 17.5K/s
  4%|▍         | 1/26 [00:00<00:05,  4.58it/s]2023-07-15 15:28:31,604 INFO Welcome to the CDS
2023-07-15 15:28:31,605 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:31,689 INFO Downloading https://download-0004-ads-clone.copernicus-climate.eu/cache-compute-0004/cache/data6/adaptor.mars.internal-1689418447.0281134-4647-12-990f4714-fc81-4762-bc1c-39b808a115e4.nc to download.nc (1.1K)
2023-07-15 15:28:31,716 INFO Downlo

In [7]:
for var in tqdm(single_slow_access):
    c.retrieve(
    'cams-global-reanalysis-eac4',
    {
        'format': 'netcdf',
        'variable': var,
        'date': '2010-01-01/2010-01-02',
        'time': '00:00',
        'area': [
            90, 179, 89,
            180,
        ],
    },
    'download.nc')
    add_var(data, var)
with open("eac4_config.yaml", "a") as file:
    yaml.dump(data, file, sort_keys=False)
data = {}

  0%|          | 0/48 [00:00<?, ?it/s]2023-07-15 15:28:36,722 INFO Welcome to the CDS
2023-07-15 15:28:36,722 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:36,813 INFO Downloading https://download-0003-ads-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.external-1689418588.8627322-5758-10-0a84e313-3868-40a2-9b58-58eee5ba68c1.nc to download.nc (1.1K)
2023-07-15 15:28:36,850 INFO Download rate 30.8K/s
  2%|▏         | 1/48 [00:00<00:11,  3.99it/s]2023-07-15 15:28:36,953 INFO Welcome to the CDS
2023-07-15 15:28:36,954 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:37,009 INFO Downloading https://download-0003-ads-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.external-1689418591.479075-5794-9-21b1d2a7-3edb-425d-b645-c3e82f2f98a9.nc to download.nc (1.1K)
2023-07-15 15:28:37,045 INFO Download

In [8]:
for var in tqdm(multi_slow_access):
    try:
        c.retrieve(
        'cams-global-reanalysis-eac4',
        {
            'format': 'netcdf',
            'variable': var,
            'date': '2010-01-01/2010-01-02',
            'time': '00:00',
            "model_level": '1'
        },
        'download.nc')
        add_var(data, var)
    except:
        print(f"Error! Skipping {var}")
        skipped.append(var)
with open("eac4_config.yaml", "a") as file:
    yaml.dump(data, file, sort_keys=False)
data = {}

  0%|          | 0/45 [00:00<?, ?it/s]2023-07-15 15:28:46,130 INFO Welcome to the CDS
2023-07-15 15:28:46,132 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:46,175 INFO Downloading https://download-0003-ads-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.external-1689427688.2990322-17746-12-cc79c31b-4b0b-4996-a172-7e4314b25ec3.nc to download.nc (455.8K)
2023-07-15 15:28:46,997 INFO Download rate 555.2K/s
  2%|▏         | 1/45 [00:00<00:43,  1.01it/s]2023-07-15 15:28:47,075 INFO Welcome to the CDS
2023-07-15 15:28:47,076 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:28:47,144 INFO Downloading https://download-0004-ads-clone.copernicus-climate.eu/cache-compute-0004/cache/data6/adaptor.mars.external-1689427691.9785323-25860-10-ea4593b7-6433-4cdd-baee-3d34970294ba.nc to download.nc (455.8K)
2023-07-15 15:28:48,116 INFO

Error! Skipping potential_vorticity


2023-07-15 15:30:17,606 INFO Request is running
2023-07-15 15:30:19,128 INFO Request is completed
2023-07-15 15:30:19,128 INFO Downloading https://download-0000-ads-clone.copernicus-climate.eu/cache-compute-0000/cache/data8/adaptor.mars.external-1689427817.6828985-22033-9-cacac16f-1182-45c3-a78f-9e915abd49bc.nc to download.nc (455.8K)
2023-07-15 15:30:20,038 INFO Download rate 501.3K/s
 76%|███████▌  | 34/45 [01:34<00:28,  2.63s/it]2023-07-15 15:30:20,133 INFO Welcome to the CDS
2023-07-15 15:30:20,133 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:30:20,161 INFO Request is queued
2023-07-15 15:30:21,183 INFO Request is running
2023-07-15 15:30:22,703 INFO Request is completed
2023-07-15 15:30:22,704 INFO Downloading https://download-0005-ads-clone.copernicus-climate.eu/cache-compute-0005/cache/data0/adaptor.mars.external-1689427821.2549765-29001-10-a6124d8d-4cf7-424e-a81f-6812ff696b63.nc to download.nc (455.8K)
2

Error! Skipping relative_humidity


2023-07-15 15:30:25,577 INFO Request is running
2023-07-15 15:30:27,110 INFO Request is completed
2023-07-15 15:30:27,110 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data2/adaptor.mars.external-1689427825.6678956-29291-4-1f1608e8-00b5-4219-8cd2-7eed5cf977ea.nc to download.nc (455.8K)
2023-07-15 15:30:27,301 INFO Download rate 2.3M/s
 82%|████████▏ | 37/45 [01:41<00:19,  2.49s/it]2023-07-15 15:30:27,455 INFO Welcome to the CDS
2023-07-15 15:30:27,455 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:30:27,493 INFO Request is queued
2023-07-15 15:30:28,543 INFO Request is running
2023-07-15 15:30:30,080 INFO Request is completed
2023-07-15 15:30:30,080 INFO Downloading https://download-0000-ads-clone.copernicus-climate.eu/cache-compute-0000/cache/data3/adaptor.mars.external-1689427828.581495-28748-2-ed582717-84d9-406e-8a6c-446ee1f94c53.nc to download.nc (455.8K)
2023-

In [9]:
# Errors

In [10]:
skipped

['potential_vorticity', 'relative_humidity']

In [11]:
c.retrieve(
'cams-global-reanalysis-eac4',
{
    'format': 'netcdf',
    'variable': 'potential_vorticity',
    'pressure_level': '1',
    'date': '2010-01-01/2010-01-02',
    'time': '00:00',
},
'download.nc')
add_var(data, 'potential_vorticity')

2023-07-15 15:31:35,714 INFO Welcome to the CDS
2023-07-15 15:31:35,714 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:31:35,819 INFO Downloading https://download-0004-ads-clone.copernicus-climate.eu/cache-compute-0004/cache/data3/adaptor.mars.external-1689427477.0520828-3315-11-abecd8ba-18b7-4758-8666-46e9e6885202.nc to download.nc (455.8K)
2023-07-15 15:31:36,356 INFO Download rate 850.8K/s


In [12]:
c.retrieve(
'cams-global-reanalysis-eac4',
{
    'format': 'netcdf',
    'variable': 'relative_humidity',
    'pressure_level': '1',
    'date': '2010-01-01/2010-01-02',
    'time': '00:00',
    'area': [
        90, 170, 80,
        180,
    ],
},
'download.nc')
add_var(data, 'relative_humidity')

2023-07-15 15:31:38,408 INFO Welcome to the CDS
2023-07-15 15:31:38,409 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-reanalysis-eac4
2023-07-15 15:31:38,503 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data2/adaptor.mars.external-1689427273.4753492-16277-9-27f8309d-e49b-4d72-b5a9-3d0943a374f0.nc to download.nc (2K)
2023-07-15 15:31:38,613 INFO Download rate 18.2K/s


In [13]:
data

{'potential_vorticity': {'factor': 1,
  'original_units': 'K m**2 kg**-1 s**-1',
  'convert_units': 'K m**2 kg**-1 s**-1',
  'cams_variables': 'pv',
  'long_names': 'Potential vorticity'},
 'relative_humidity': {'factor': 1,
  'original_units': '%',
  'convert_units': '%',
  'cams_variables': 'r',
  'long_names': 'Relative humidity'}}

In [14]:
with open("eac4_config.yaml", "a") as file:
    yaml.dump(data, file, sort_keys=False)
data = {}